In [12]:
import os
import re

import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
tqdm.pandas()

In [13]:
valid_CMA = [
    "St. John's", 
    "Halifax", 
    "Québec", 
    "Montréal", 
    "Ottawa - Gatineau (Ontario part / partie de l'Ontario)", 
    "Toronto", 
    "Kitchener - Cambridge - Waterloo", 
    "Hamilton", 
    "London", 
    "Winnipeg", 
    "Regina", 
    "Saskatoon", 
    "Edmonton", 
    "Calgary", 
    "Vancouver", 
    "Victoria"
]

valid_mont = [
    "Montréal", 
    "Montréal-Est", 
    "Montréal-Ouest", 
    "Westmount", 
    "Côte-Saint-Luc", 
    "Mont-Royal", 
    "Hampstead", 
    "Dorval", 
    "Pointe-Claire", 
    "Dollard-Des Ormeaux", 
    "Kirkland", 
    "Beaconsfield", 
    "Baie-D'Urfé", 
    "Sainte-Anne-de-Bellevue", 
    "Senneville"
]

Filter for the CMAs that we are interested in, and replace the Toronto CMA with the Greater Toronto Area region

In [39]:
gdf_cma = gpd.read_file('../data/boundaries/cen_cma/')

In [ ]:
gdf_regions = gdf_cma.loc[gdf_cma['CMANAME'].isin(valid_CMA)].reset_index(drop=True)

# Substitute Toronto CMA for GTA
gdf_gta = gpd.read_file('../data/boundaries/greater-toronto-area.gpkg')
gdf_gta = gdf_gta.to_crs(gdf_regions.crs)
gdf_gta['CMANAME'] = 'Greater Toronto Area'
gdf_regions = pd.concat([gdf_regions, gdf_gta], ignore_index=True)
gdf_regions = gdf_regions[gdf_regions['CMANAME'] != 'Toronto']

gdf_regions = gdf_regions.rename(columns={'CMANAME': 'REGION_NAME'})
gdf_regions = gdf_regions.drop(columns=['DGUID', 'DGUIDP', 'CMATYPE', 'LANDAREA', 'PRUID'])
gdf_regions.to_file('../data/boundaries/valid_regions.gpkg', driver='GPKG')

Filter for the CSDs which are contained within a CMA/region, then replace all the CSDs on the Island of Montreal with the combined geography

In [40]:
gdf_csd = gpd.read_file('../data/boundaries/cen_csd/')

,CSDUID,DGUID,CSDNAME,CSDTYPE,LANDAREA,PRUID,geometry
0,1001101,2021A00051001101,"Division No. 1, Subd. V",SNO,870.8928,10,"MULTIPOLYGON (((8991051.954 2038839.069, 89910..."
1,1001105,2021A00051001105,Portugal Cove South,T,1.0770,10,"POLYGON ((9001504.369 2049856.597, 9001184.977..."
2,1001113,2021A00051001113,Trepassey,T,54.2130,10,"POLYGON ((8992311.551 2054468.074, 8994001.626..."
3,1001120,2021A00051001120,St. Shott's,T,1.0729,10,"POLYGON ((8985240.566 2028560.540, 8984967.974..."
4,1001124,2021A00051001124,"Division No. 1, Subd. U",SNO,742.3781,10,"MULTIPOLYGON (((8995265.971 2098624.091, 89952..."
